In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz, process


/root/venv/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
df = pd.read_csv('ZaraData.csv')

In [ ]:
df.head()

name               sku brand  \
0  SATIN EFFECT CORSET BODYSUIT   128666521-966-2  ZARA   
1              LONG PRINT DRESS   142448983-701-1  ZARA   
2             PLAID SHIRT DRESS  138550324-800-18  ZARA   
3                  CORDUROY CAP   131213214-807-3  ZARA   
4      CLASSIC STRIPED SWIMSUIT   120689725-401-3  ZARA   

                                         description  price availability  \
0  Bodysuit with sweetheart neckline and adjustab...   29.9      InStock   
1  Long dress with round neck and long sleeves. B...   49.9        empty   
2  Shirtdress with lapel collar and long sleeves....   27.9        empty   
3               Cap with visor. Back fit adjustment.   25.9      InStock   
4  Swimsuit made of lightweight technical fabric....   35.9      InStock   

      condition                             color  \
0  NewCondition        Color Pale pink | 0219/805   
1  NewCondition            Color Brick | 9329/912   
2  NewCondition            Color Black | 1718/552   
3  NewCondition  Color Anthracite grey | 9065/325   
4  NewCondition        Color Navy blue | 0495/312   

                                           size_list  event_category category  \
0                                              S/M/L           empty     HOME   
1  XS/XS/ -/ /We'll let you know when the item is...  Formal, Casual    DRESS   
2  12-18 months (33,9 INCHES)/18-24 months (36,2 ...  Formal, Casual    DRESS   
3                                              empty  Casual, Sporty      CAP   
4                                             M/L/XL           empty     HOME   

     complimentary_clothing_category scope_(inscope/outofscope)  \
0                              empty                 OUTOFSCOPE   
1      Shoes, Sandals, Bags, Jewelry                    INSCOPE   
2      Shoes, Sandals, Bags, Jewelry                    INSCOPE   
3  T-Shirt, Jacket, Shorts, Sneakers                    INSCOPE   
4                              empty                 OUTOFSCOPE   

                                                 url  
0  https://www.zara.com/us/en/satin-effect-corset...  
1  https://www.zara.com/us/en/long-print-dress-p0...  
2  https://www.zara.com/us/en/plaid-shirt-dress-p...  
3  https://www.zara.com/us/en/corduroy-cap-p09065...  
4  https://www.zara.com/us/en/classic-striped-swi...

In [ ]:
# change evento_clothing_mapping -> delete categories

data = df
# Create a DataFrame from the mock data
clothing_data = pd.DataFrame(data)

# Mapping of event categories to clothing categories and their complimentary categories

event_to_clothing_mapping = {
    'Casual and Everyday Wear': {
        'categories': ['T-SHIRT', 'JEANS', 'SWEATSHIRT', 'LEGGINGS', 'SHOES', 'TOP', 'PANTS', 'SHIRT', 'SHORTS', 'SLIPPERS'],
        'complimentary': ['Shoes', 'Sandals', 'Bags'],
    },
    'Formal and Business Events': {
        'categories': ['BLAZER', 'DRESS PANTS', 'BLOUSE', 'PENCIL SKIRT', 'DRESS SHIRT', 'DRESS SHOES', 'BELT', 'SHOES', 'DRESS'],
        'complimentary': ['DRESS SHOES', 'BELT', 'JACKET'],
    },
    'Evening and Party Wear': {
        'categories': ['DRESS', 'PARTY TOPS', 'DRESSY SKIRTS', 'HIGH HEELS', 'CLUTCH BAGS', 'STATEMENT JEWELRY', 'ELEGANT JUMPSUITS'],
        'complimentary': ['BELT', 'STATEMENT JEWELRY', 'HIGH HEELS'],
    },
    'Beach and Vacation': {
        'categories': ['BIKINIS', 'COVER-UPS', 'SUN DRESSES', 'SANDALS', 'SUN HATS', 'BEACH BAGS', 'SUNGLASSES', 'SWIMSUITS', 'FLIP-FLOPS'],
        'complimentary': ['SUN HATS', 'SUNGLASSES', 'SANDALS'],
    },
    'Athletic and Workout': {
        'categories': ['SPORTS LEGGINGS', 'SPORTS BRAS', 'TANK TOPS', 'JOGGERS', 'ATHLETIC SHORTS', 'ATHLETIC SHOES', 'PERFORMANCE TOPS', 'ACTIVE JACKETS'],
        'complimentary': ['SHOES', 'SPORTS BRAS', 'SOCKS'],
    },
    'Sleep and Loungewear': {
        'categories': ['PAJAMA SETS', 'NIGHTGOWNS', 'SLEEP SHIRTS', 'LOUNGE PANTS', 'SLIPPERS', 'ROBES', 'LOUNGE TOPS'],
        'complimentary': ['SLIPPERS', 'ROBES', 'SOCKS'],
    },
    'Winter and Cold Weather': {
        'categories': ['COATS', 'SWEATERS', 'SCARVES', 'BEANIES', 'GLOVES', 'SNOW BOOTS', 'THERMAL LEGGINGS', 'PARKAS'],
        'complimentary': ['BEANIES', 'GLOVES', 'SCARVES'],
    },
    'Summer and Hot Weather': {
        'categories': ['SUNDRESSES', 'SHORTS', 'TANK TOPS', 'SANDALS', 'SUNGLASSES', 'SUN HATS', 'LIGHTWEIGHT CARDIGANS', 'ROMPERS'],
        'complimentary': ['SUNGLASSES', 'SUN HATS', 'SANDALS'],
    },
    'Festive and Holiday Wear': {
        'categories': ['DRESS', 'SPARKLY TOPS', 'FESTIVE SWEATERS', 'PARTY ACCESSORIES', 'FESTIVE JEWELRY'],
        'complimentary': ['PARTY ACCESSORIES', 'FESTIVE JEWELRY'],
    },
}




In [ ]:
# ... (previous code)



def recommend_outfit(event):
    matched_event, score = process.extractOne(event.lower(), event_to_clothing_mapping.keys(), scorer=fuzz.partial_ratio)
    
    if score >= 80:  # You can adjust the threshold as needed
        event_info = event_to_clothing_mapping.get(matched_event)
        recommended_items = clothing_data[clothing_data['category'].isin(event_info['categories'])]
        complimentary_items = clothing_data[clothing_data['complimentary_clothing_category'].apply(lambda x: any(cat in x for cat in event_info['complimentary']))]
        
        recommended_item = recommended_items.sample(n=1)
        complimentary_item = complimentary_items.sample(n=2)
        
        return recommended_item, complimentary_item
    else:
        return None, "Sorry, I couldn't find suitable recommendations for that event."

    

In [ ]:
# Example interaction
user_event = input("Please enter your event occasion: ")
recommendation, upsell = recommend_outfit(user_event)

if recommendation is not None:
    print("Here is your outfit recommendation:")
    print(recommendation[['name', 'price', 'description']].to_string(index=False))
    
    if upsell is not None:
        print("\nYou might also like this complimentary item:")
        print(upsell[['name', 'price', 'description']].to_string(index=False))
    else:
        print("Sorry, there is no complimentary item available.")
else:
    print(upsell)

KeyboardInterrupt: Interrupted by user

In [ ]:
def customer_menu():
    while True:
        print("Welcome, Customer! What would you like help with today?")
        print("1. Recommendations")
        print("2. Can't find item in stock")
        print("3. Event-based recommendation")
        print("4. Get customer data")
        print("5. Influencer Collaboration")
        print("6. Exit")
        choice = input("Enter your choice (1/2/3/4/5/6): ")

        if choice == "1":
            recommendations()
        elif choice == "2":
            find_item_in_stock()
        elif choice == "3":
            event_based_recommendation()
        elif choice == "4":
            get_customer_data()
        elif choice == "5":
            influencer_collaboration()
        elif choice == "6":
            print("Exiting customer menu.")
            break
        else:
            print("Invalid choice.")

In [ ]:
def influencer_collaboration():
    print("Influencer Collaboration:")
    
    # Read data from "insta_data.csv" using pandas
    try:
        influencer_data = pd.read_csv("insta_data.csv")
    except FileNotFoundError:
        print("Error: Could not find 'insta_data.csv'. Make sure the file is in the same directory.")

    print("Influencer data loaded successfully.")

    instagram_name = input("Enter your Instagram name: ")
    user_influence = influencer_data[influencer_data["channel_info"] == instagram_name]

    score = int(user_influence["influence_score"])
    
    if user_influence.empty:
        print(f"Sorry, {instagram_name} not found in the influencer data.")
        return
    
    influence_score = int(user_influence["influence_score"].values[0])
    country=user_influence["country"].values[0]
      
    
    if influence_score>70:
    
        season = get_season(country)
        if season:
            collaboration_options = {
                "summer": "Share your creation on Instagram with the hashtag #SummerVibesChallenge and tag @OurFashionBot. Don't forget to mention why your outfit represents the perfect summer vibe.",
                "winter": "Show off your winter style! Use the hashtag #WinterFashionGlow and mention @OurFashionBot to join our winter fashion campaign.",
                "spring": "Spring into fashion! Tag @OurFashionBot in your spring-themed outfit post with the hashtag #SpringFreshLooks.",
                "fall": "Fall in love with fashion! Share your fall outfit using #FallFashionFaves and tag @OurFashionBot to be featured."
            }
            
            if season in collaboration_options:
                print(f"Congrats I see you qulaify for our collaboration program. I see the season in {country} is currently {season}")
                
                print("Collaboration Suggestion:")
                print(collaboration_options[season])
            else:
                print("No collaboration suggestion available for this season.")

In [ ]:
def get_season(country):
    # Search for season information on Google
    search_query = f"current weather in {country}"
    search_url = f"https://www.google.com/search?q={search_query}"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    
    response = requests.get(search_url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        result = soup.find("div", class_="BNeawe iBp4i AP7Wnd")
        
        if result:
            temperature = (result.get_text().lower())
            temperature = int(temperature.split('°')[0])
            if temperature < 10:
                season = "winter"
            elif 10 <= temperature < 15:
                season = "spring"
            elif 15 <= temperature < 20:
                season = "fall"
            else:
                season = "summer"
            return season
        else:
            print("Could not retrieve season information from Google.")
            return None
    else:
        print("Error: Unable to connect to Google.")
        return None

In [ ]:
def customer_menu():
    while True:
        print("Welcome, Customer! What would you like help with today?")
        print("1. Recommendations")
        print("2. Can't find item in stock")
        print("3. Event-based recommendation")
        print("4. Get customer data")
        print("5. Influencer Collaboration")
        print("6. Exit")
        choice = input("Enter your choice (1/2/3/4/5/6): ")
        
        if choice == "5":
            influencer_collaboration()
        elif choice == "6":
            print("Exiting customer menu.")
            break
        else:
            print("Invalid choice.")

In [ ]:
#Main function
def main():
    while True:
        print("Hi! Welcome to the Zara chatbot. Before we begin, please choose if you are an employee or customer.")
        print("1. Employee")
        print("2. Customer")
        print("3. Exit")
        choice = input("Enter your choice (1/2/3): ")

        if choice == "1":
            employee_id = input("Please provide your employee ID for verification: ")
            employee_menu() #function call

        elif choice == "2":
            customer_menu() #function call

        elif choice == "3":
            print("Goodbye! Have a great day.")
            break
    
        else:
            print("Invalid choice. Please select '1' for Employee, '2' for Customer, or '3' to Exit.")

       
if __name__ == "__main__":
    main()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=efa188a0-2f94-4804-9f19-625dc30c6533' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>